In [18]:
from base_dd import BDD_EVENTS
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

TOUT SPORT AVEC TOUT LES VARIABLES

In [19]:
variable_qanti = ['Age', 'Height', 'Weight']
variable_quali = ['NOC', 'Sex']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
if dfa_0.shape[0] > dfa_1.shape[0]:
    dfa_0_sampled = dfa_0.sample(n=len(dfa_1))
    dfa_1_sampled = dfa_1.copy()
else:
    dfa_1_sampled = dfa_1.sample(n=len(dfa_0))
    dfa_0_sampled = dfa_0.copy()
dfa = pd.concat([dfa_1_sampled, dfa_0_sampled], axis=0)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [20]:
modelaa = LogisticRegression(solver='newton-cg')
modelaa.fit(X, Y)
print(modelaa.coef_)
print(modelaa.score(X, Y))
print(modelaa.score(XG, YG))

[[ 0.01442912  0.01861488  0.01157901 -0.74391759 -0.79373604 -0.29280533
  -1.18774221 -1.74476353 -1.0771144   1.38423146  0.85344312  0.55726279
  -0.63252235 -0.45752425  1.50960413  0.68697144  1.52720017  0.97214187
  -0.54695422 -1.19674984  0.13935977  0.17677994 -0.73099216 -1.02423165
  -0.42939282 -1.22183462 -0.84156845  0.52922479  0.         -1.07192608
  -0.60037039  1.1908165  -0.20218551 -0.14785823  1.04902115 -0.61262241
  -0.69773222 -0.61638933  1.1783522  -0.91172084 -0.89121813 -0.49751636
   0.14036953  1.72158444 -0.61972097  0.51121044 -0.80705675 -0.59333587
  -0.26029791 -0.24928445 -0.34325657 -0.68316265  1.1900141   0.
   1.56608316 -1.22062666  0.45830966  1.24091923 -0.03467057 -0.34112846
  -0.37505108 -1.13250637 -1.271104   -0.102767   -1.26255007  0.90347692
  -0.10687568  1.66850551  2.2785084   0.2114145   1.49407075  1.14740126
   1.53687025 -0.34732112 -0.42170816 -0.52265369  1.18114739 -0.34591765
   2.63840474  0.96112017 -0.36421908  1.73481

In [21]:
model = LogisticRegression(solver='newton-cg')
model.fit(Xa, Ya)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[ 0.01727721  0.01441341  0.01337217 -0.57729984 -0.83234333 -0.16550392
  -1.14601339 -1.74114758 -1.2732974   1.38882237  0.8912644   0.46993426
  -0.79908387 -0.54329663  1.51187246  0.72459053  1.52051332  1.18887721
  -0.66767124 -1.24714486  0.00436505  0.23876252 -0.93444134 -1.26550998
  -0.80783466 -1.41403405 -0.90249167  0.51705427  0.         -1.25906805
  -0.54604988  1.22512001 -0.21858545 -0.42508907  1.06186688 -0.33037069
  -0.66359666 -0.6506511   1.22724499 -0.92254449 -0.56013613 -0.72206135
   0.17930155  1.84876787 -0.86886136  0.51549382 -0.94188021 -0.92067803
  -0.37622296 -0.48498084  0.         -0.6296328   1.16811191  0.
   1.5837549  -1.29114385  0.50253202  1.23563412 -0.0917339  -0.35794037
  -0.03361653 -0.98647539 -1.13165198  0.10281417 -1.3098088   0.95843205
  -0.21069449  2.02826377  2.33126586  0.41637757  1.54914606  1.20245964
   1.47784199 -0.50840672 -0.06420526 -0.8244601   1.24286353 -0.33348608
   2.68071474  0.92513938 -0.317724    1.78200

BASKET AVEC TOUTE LES VARIABLES

In [22]:
variable_qanti = ['Age', 'Height', 'Weight']
variable_quali = ['NOC', 'Sex']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
df = df[BDD_EVENTS['Sport'].isin(['Basketball'])]
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
if dfa_0.shape[0] > dfa_1.shape[0]:
    dfa_0_sampled = dfa_0.sample(n=len(dfa_1))
    dfa_1_sampled = dfa_1.copy()
else:
    dfa_1_sampled = dfa_1.sample(n=len(dfa_0))
    dfa_0_sampled = dfa_0.copy()
dfa = pd.concat([dfa_1_sampled, dfa_0_sampled], axis=0)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [23]:
modelaa = LogisticRegression(solver='newton-cg')
modelaa.fit(X, Y)
print(modelaa.coef_)
print(modelaa.score(X, Y))
print(modelaa.score(XG, YG))

[[-1.79054221e-02  7.31122294e-03 -2.53743774e-04  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.46958568e+00
   0.00000000e+00  0.00000000e+00  2.06383701e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.02615648e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.19747710e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  8.99384435e-01
   0.00000000e+00  0.00000000e+00  1.34436984e+00  0.00000000e+00
  -5.09188776e-01  0.00000000e+00 -1.87348694e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.27088405e-01 -2.50315196e-02
   0.00000000e+00  0.00000000e+00 -8.44482731e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   8.86539580e-01  0.00000000e+00  8.93725006e-02  0.00000000e+00
  -1.52883058e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [24]:
model = LogisticRegression(solver='newton-cg')
model.fit(Xa, Ya)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[-3.03915947e-02  1.85098192e-03  3.25853731e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.26832387e+00
   0.00000000e+00  0.00000000e+00  2.12413556e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.01762339e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.31380259e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  9.14056153e-01
   0.00000000e+00  0.00000000e+00  1.19728027e+00  0.00000000e+00
  -3.28093015e-01  0.00000000e+00 -1.66985050e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.94425447e-02
   0.00000000e+00  0.00000000e+00 -4.84979866e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.41702316e+00  0.00000000e+00  1.67134663e-01  0.00000000e+00
  -1.51424976e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

BASKET AVEC NOC

In [34]:
variable_qanti = []
variable_quali = ['NOC']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
df = df[BDD_EVENTS['Sport'].isin(['Basketball'])]
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
if dfa_0.shape[0] > dfa_1.shape[0]:
    dfa_0_sampled = dfa_0.sample(n=len(dfa_1))
    dfa_1_sampled = dfa_1.copy()
else:
    dfa_1_sampled = dfa_1.sample(n=len(dfa_0))
    dfa_0_sampled = dfa_0.copy()
dfa = pd.concat([dfa_1_sampled, dfa_0_sampled], axis=0)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [35]:
modelaa = LogisticRegression(solver='newton-cg')
modelaa.fit(X, Y)
print(modelaa.coef_)
print(modelaa.score(X, Y))
print(modelaa.score(XG, YG))

[[ 0.          0.          0.          0.         -1.40079667  0.
   0.          1.43751701  0.          0.          0.          1.26110585
   0.          0.          0.          0.          0.          0.
  -0.99757862  0.          0.          0.          0.          0.
  -0.7903496   0.          0.          0.          1.25150859  0.
   0.          1.48184005  0.         -0.54548447  0.         -0.38642594
   0.          0.          0.         -1.11936535  0.3233211   0.
   0.         -0.54548447  0.          0.          0.          0.
   0.          1.04528205  0.          0.27147828  0.         -1.02659969
   0.          0.          0.          0.          0.         -1.44887215
   0.          0.          1.6803697  -0.41206055  0.          2.13925617
   0.         -0.81035604  1.39848608 -0.82976281  0.          0.
   0.         -1.04065433  0.          0.          0.          0.
  -0.99757862  0.         -1.30323574  0.          0.          0.
   0.          0.          0.       

In [36]:
model = LogisticRegression(solver='newton-cg')
model.fit(Xa, Ya)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[ 0.          0.          0.          0.         -1.34963107  0.
   0.          1.47381367  0.          0.          0.          1.17392061
   0.          0.          0.          0.          0.          0.
  -1.00949636  0.          0.          0.          0.          0.
  -0.91624699  0.          0.          0.          1.18154022  0.
   0.          1.51947899  0.         -0.42534247  0.         -0.39923272
   0.          0.          0.         -1.16546899  0.2378311   0.
   0.         -0.31206231  0.          0.          0.          0.
   0.          1.24807632  0.          0.35626262  0.         -0.86445804
   0.          0.          0.          0.          0.         -1.37621928
   0.          0.          1.56347765 -0.31206231  0.          2.05535566
   0.         -0.80844835  1.30937095 -0.60578428  0.          0.
   0.         -1.05183753  0.          0.          0.          0.
  -0.96443056  0.         -1.40186657  0.          0.          0.
   0.          0.          0.       

BASKET SANS NOC

In [28]:
variable_qanti = ['Age', 'Height', 'Weight']
variable_quali = ['Sex']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
df = df[BDD_EVENTS['Sport'].isin(['Basketball'])]
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
if dfa_0.shape[0] > dfa_1.shape[0]:
    dfa_0_sampled = dfa_0.sample(n=len(dfa_1))
    dfa_1_sampled = dfa_1.copy()
else:
    dfa_1_sampled = dfa_1.sample(n=len(dfa_0))
    dfa_0_sampled = dfa_0.copy()
dfa = pd.concat([dfa_1_sampled, dfa_0_sampled], axis=0)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [29]:
modelaa = LogisticRegression(solver='newton-cg')
modelaa.fit(X, Y)
print(modelaa.coef_)
print(modelaa.score(X, Y))
print(modelaa.score(XG, YG))

[[-0.01630304  0.01760504  0.00749257 -0.74096146]]
0.727917121046892
0.003


In [30]:
model = LogisticRegression(solver='newton-cg')
model.fit(Xa, Ya)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[-0.01907404  0.0187268   0.00767685 -0.81405124]]
0.5627044711014176
0.56


ATHLETICS

In [31]:
variable_qanti = ['Age', 'Height', 'Weight']
variable_quali = ['NOC', 'Sex']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
df = df[BDD_EVENTS['Sport'].isin(['Athletics'])]
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
if dfa_0.shape[0] > dfa_1.shape[0]:
    dfa_0_sampled = dfa_0.sample(n=len(dfa_1))
    dfa_1_sampled = dfa_1.copy()
else:
    dfa_1_sampled = dfa_1.sample(n=len(dfa_0))
    dfa_0_sampled = dfa_0.copy()
dfa = pd.concat([dfa_1_sampled, dfa_0_sampled], axis=0)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [32]:
modelaa = LogisticRegression(solver='newton-cg')
modelaa.fit(X, Y)
print(modelaa.coef_)
print(modelaa.score(X, Y))
print(modelaa.score(XG, YG))

[[-1.25781370e-02  2.20084160e-03  4.43957529e-03 -2.75464069e-01
  -2.27395538e-01  7.66216157e-01 -2.14720329e-01 -4.45041035e-01
  -9.59675407e-01  5.50218265e-01 -1.06696740e-01 -3.31667607e-01
  -1.80239837e-01 -1.92544813e-01  1.08314713e+00  2.85998050e-02
  -3.75272212e-01  1.64091290e+00 -4.41381090e-01 -6.25072092e-01
   4.90206719e-01  4.88849637e-01 -4.68741841e-01 -6.09721295e-01
   0.00000000e+00 -3.70293121e-01 -5.54914565e-01  7.78899430e-01
   0.00000000e+00 -5.49557235e-01 -4.10466825e-01  6.78106699e-01
   3.18856752e-01 -1.09856455e-01  7.25630399e-01 -3.95106366e-01
  -3.69033470e-01 -3.01308868e-01  1.19953190e+00 -3.24914408e-01
  -6.95822297e-01 -4.73748197e-01 -4.67515599e-01  7.20933446e-01
  -4.96716977e-01 -1.28687180e-01 -3.98936309e-01 -2.69789242e-01
  -4.78386990e-01 -2.33583416e-01 -2.10648130e-01 -5.27511177e-01
   7.27204074e-01  0.00000000e+00  1.52967868e+00 -7.99283198e-01
   6.99962247e-01  2.65806693e-01  1.42586683e-01 -2.85248433e-01
   2.44074

In [33]:
model = LogisticRegression(solver='newton-cg')
model.fit(Xa, Ya)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[ 4.75814327e-04  1.30252077e-03  7.31148925e-03 -1.71026543e-01
  -4.33934499e-01  6.14579069e-01 -3.00744805e-01 -6.49971740e-01
  -1.05619351e+00  6.65649290e-01 -2.55025659e-02 -4.29660516e-01
  -3.11844848e-01  0.00000000e+00  1.22354631e+00  1.16552585e-01
  -3.28387692e-01  1.71790863e+00 -2.87648608e-01 -7.04955812e-01
   5.73042813e-01  4.81988988e-01 -7.47184983e-01 -6.68603857e-01
   0.00000000e+00  0.00000000e+00 -2.97444721e-01  6.04066245e-01
   0.00000000e+00 -4.92092641e-01 -5.64769120e-02  8.80154435e-01
   2.61314621e-01  0.00000000e+00  1.01339364e+00 -5.21552562e-01
  -3.18739541e-01 -4.07615168e-01  1.31942536e+00 -6.16956406e-01
  -8.28885451e-01 -4.14414558e-01 -4.82542835e-01  9.52392049e-01
  -4.71574285e-01  3.85573301e-01 -4.12532623e-01 -5.41289729e-01
  -7.47434214e-01 -1.56586658e-01 -3.00068293e-01 -6.07342858e-01
   8.22253128e-01  0.00000000e+00  1.43210829e+00 -6.59542634e-01
   7.68524158e-01  2.27002576e-01  5.65567423e-02 -1.59996365e-01
   3.22162